In [ ]:
import torch
import random
import faulthandler

from IPython.display import clear_output
from vctr.models.lstm.defaults import SEQUENCE_LENGTH
from vctr.models.lstm.lstm_model import HybridModel
from vctr.models.lstm.main import VCNet
from vctr.models.lstm.utils import get_new_model_name
from vctr.models.lstm.actions import new_model, save_model, train_model, load_model
from vctr.trading.coins import trainable_coins

faulthandler.enable()
torch.autograd.set_detect_anomaly(True)

hidden_size = 32
num_layers = 1
num_classes = 3
sequence_length = SEQUENCE_LENGTH

# model = HybridModel(
#     num_features=379,
#     num_classes=3,
#     hidden_size=128,
#     out_channels=256,
#     num_layers=2,
#     kernel_size=3,
#     use_conv=True,
#     bidirectional=True,
# )
model = VCNet(input_size=320, hidden_size=256, num_classes=3, num_layers=2)
model.name = get_new_model_name()
# model_name = 'lstm-mk-189'
# model = load_model('lstm-mk-649-COTI-XTZ-LOOM-LRC-BNT')

trn_params = []

coins = random.sample(trainable_coins, 5)

for coin in ['REN']:
    clear_output()
    model.name = f'{model.name}-{coin}'
    print(f'Training on {coin}')
    print(f'Current model: {model.name}')
    train_model(
        model=model,
        epochs=200,
        symbol=coin,
        timeframes=['15m'],
        batch_size=512,
        learning_rate=0.00001,
        label_args=(0.03, 0.005),
        test_pct=0.3,
        sequence_length=sequence_length,
        find_optimal_lr=False,
    )

    save_model(model, model.name)
    save_model(model, 'latest')


In [ ]:
import torch
import random
import faulthandler

from IPython.display import clear_output
from vctr.models.lstm.defaults import SEQUENCE_LENGTH
from vctr.models.lstm.utils import get_new_model_name
from vctr.models.lstm.actions import save_model
from vctr.models.lstm.genetic import run_genetic_algorithm
from vctr.trading.coins import trainable_coins
from vctr.models.lstm.main import VCNet
from vctr.models.lstm.lstm_model import HybridModel
from bayes_opt import BayesianOptimization

faulthandler.enable()
torch.autograd.set_detect_anomaly(True)
test_pct = 0.3

label_args = (0.05, 0.005)
cxpb = 0.5
mutpb = 0.3
population_size = 100


def model_evaluation(hidden_size, out_channels, num_layers, kernel_size, bidirectional):
    hidden_size = int(hidden_size)
    out_channels = int(out_channels)
    num_layers = int(num_layers)
    kernel_size = int(kernel_size)
    bidirectional = bool(round(bidirectional))

    model = HybridModel(
        num_features=486,
        num_classes=3,
        bidirectional=bidirectional,
        hidden_size=hidden_size,
        out_channels=out_channels,
        num_layers=num_layers,
        kernel_size=kernel_size,
    )
    model.name = get_new_model_name()

    coins = random.sample(trainable_coins, 5)

    for coin in ['RNDR']:  # ['REN']:
        clear_output()
        model.name = f'{model.name}-{coin}'
        print(f'Training on {coin}')
        print(f'Current model: {model.name}')
        fitness_score = run_genetic_algorithm(
            model=model,
            symbol=coin,
            timeframes=['30m'],
            batch_size=512,
            sequence_length=SEQUENCE_LENGTH,
            label_args=label_args,
            test_pct=test_pct,
            cxpb=cxpb,
            population_size=population_size,
            mutpb=mutpb,
            ngen=5,  # Change to 5 generations
            # start='2021-01-01',
            # end='2021-02-01',
        )

        save_model(model, model.name)
        save_model(model, 'latest')

    return fitness_score


# Define the search space for hyperparameters
pbounds = {
    'hidden_size': (4, 512),
    'out_channels': (4, 128),
    'num_layers': (1, 4),
    'kernel_size': (1, 5),
    'bidirectional': (0, 1),
}

optimizer = BayesianOptimization(
    f=model_evaluation,
    pbounds=pbounds,
    random_state=42,
)

# Optimize the hyperparameters
optimizer.maximize(
    init_points=5,
    n_iter=10,
)

# Get the best hyperparameters
best_params = optimizer.max['params']

# Convert the float values of the best_params dictionary to their appropriate types
best_params['hidden_size'] = int(best_params['hidden_size'])
best_params['out_channels'] = int(best_params['out_channels'])
best_params['num_layers'] = int(best_params['num_layers'])
best_params['kernel_size'] = int(best_params['kernel_size'])
best_params['bidirectional'] = bool(round(best_params['bidirectional']))

print('Best hyperparameters found:', best_params)


In [1]:
import torch
import random
import faulthandler

from IPython.display import clear_output
from vctr.models.lstm.defaults import SEQUENCE_LENGTH
from vctr.models.lstm.utils import get_new_model_name
from vctr.models.lstm.actions import save_model, load_model
from vctr.models.lstm.genetic import run_genetic_algorithm
from vctr.trading.coins import trainable_coins
from vctr.models.lstm.lstm_model import HybridModel
from vctr.models.lstm.main import VCNet

faulthandler.enable()
torch.autograd.set_detect_anomaly(True)
test_pct = 0.3

# @ChatGPT Find the best hyperparameters for the genetic algorithm
cxpb = 0.7
mutpb = 0.6
population_size = 100
ngen = 50
label_args = (0.07, 0.01)
hidden_size = 64
out_channels = 51
num_layers = 5
kernel_size = 3
bidirectional = True
use_conv = True
dropout = 0.2

model = HybridModel(
    num_features=297,
    num_classes=3,
    bidirectional=bidirectional,
    hidden_size=hidden_size,
    out_channels=out_channels,
    num_layers=num_layers,
    kernel_size=kernel_size,
    use_conv=use_conv,
)
# model = VCNet(input_size=379, hidden_size=8, num_classes=3, num_layers=2)

# model = load_model('latest')
model.name = get_new_model_name()

coins = random.sample(trainable_coins, 5)

for coin in ['TRX']:
    clear_output()
    model.name = f'{model.name}-{coin}'
    print(f'Training on {coin}')
    print(f'Current model: {model.name}')
    run_genetic_algorithm(
        model=model,
        symbol=coin,
        timeframes=['30m'],
        batch_size=256,
        sequence_length=SEQUENCE_LENGTH,
        label_args=label_args,
        test_pct=test_pct,
        cxpb=cxpb,
        population_size=population_size,
        mutpb=mutpb,
        ngen=ngen,
    )

    save_model(model, model.name)
    save_model(model, 'latest')


Training on TRX
Current model: lstm-mk-1293-TRX
